In [1]:
import os
import yaml
import pandas as pd
from copy import deepcopy
from yaml import Dumper, Loader
from random import randint

from CORAL import SharedLibrary, GlobalManager

with open("turbine_install.yaml", "r") as f:
    config = yaml.load(f, Loader=Loader)

In [2]:
config  # Shared resources are flagged with the '_shared_pool_' input.

{'design_phases': [],
 'feeder': '_shared_pool_',
 'install_phases': ['TurbineInstallation'],
 'num_feeders': 2,
 'plant': {},
 'site': {'depth': 20, 'distance': 40},
 'turbine': '12MW_generic',
 'wtiv': '_shared_pool_'}

In [24]:
library_path = os.path.join(os.getcwd(), "library")

allocations = {                           # Allocations are defined per input field
    "vessels": {
        "wtiv": ('example_wtiv', 2),      # This is currently just copying 'example_wtiv' N times but could be expanded to have
        "feeder": ('example_feeder', 6)   # a list of unique vessels. 
    },
    
#     "ports": {
#         "port": ('shared_port', 3)      # To make shared ports work, go to the bottom and uncomment "port": "_shared_pool_", save and restart notebook.
#     }
}

In [25]:
configs = [deepcopy(config) for _ in range(10)]

for i, config in enumerate(configs):   # Creating 10 of the same project with anywhere between 35 and 45 turbinesm starting at 1000, 2000, etc.
    
    config['project_start'] = i * 1000
    config["plant"]["num_turbines"] = randint(35, 45)

In [26]:
manager = GlobalManager(configs, allocations, library_path=library_path)
manager.setup()
manager.run()

In [27]:
pd.DataFrame(manager.logs)      # request-vessel is when the project's request to use one of those vessels was successful.
                                # delay-vessel is when that request succeeded - project initialization
                                # Initialized: when the project was supposed to start
                                # Started: when it actually did. Maximum of the request- columns

,name,Initialized,Started,Finished,request-feeder,request-wtiv,delay-feeder,delay-wtiv
0,Project 1,0,0.000000,1861.433333,0,0.000000,0,0.000000
1,Project 2,1000,1000.000000,2913.793333,1000,1000.000000,0,0.000000
2,Project 3,2000,2000.000000,4227.953333,2000,2000.000000,0,0.000000
3,Project 4,3000,3000.000000,4913.793333,3000,3000.000000,0,0.000000
4,Project 5,4000,4227.953333,6298.826667,4000,4227.953333,0,227.953333
5,Project 6,5000,5000.000000,7018.513333,5000,5000.000000,0,0.000000
6,Project 7,6000,6298.826667,8579.140000,6000,6298.826667,0,298.826667
7,Project 8,7000,7018.513333,8932.306667,7000,7018.513333,0,18.513333
8,Project 9,8000,8579.140000,10911.813333,8000,8579.140000,0,579.140000
9,Project 10,9000,9000.000000,11070.873333,9000,9000.000000,0,0.000000


In [5]:
config = {
    'wtiv': '_shared_pool_',
    'feeder': '_shared_pool_',
    'num_feeders': 2,
    'site': {'depth': 20, 'distance': 40},
    "plant": {},
#     'port': "_shared_pool_",
    'turbine': "12MW_generic",
    
    "design_phases": [],
    "install_phases": ["TurbineInstallation"]
}

with open("turbine_install.yaml", "w") as f:
    yaml.dump(config, f, Dumper=Dumper)